In [2]:
%pwd

'/home/visa'

In [1]:
import xgboost

In [12]:
import os
os.chdir("/home/visa/Python")

In [13]:
import pandas as pd 
import xgboost as xgb
from pandas import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

In [14]:
allstate = pd.read_csv("Purchase.csv")

In [15]:
allstate.head()

,QuoteNumber,Original_Quote_Date,QuoteConversion_Flag,Field6,Field7,Field8,Field9,Field10,Field11,Field12,...,GeographicField59A,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,1,2013-08-16,0,B,23,0.9403,0.0006,965,1.0200,N,...,9,9,-1,8,-1,18,-1,10,N,CA
1,2,2014-04-22,0,F,7,1.0006,0.0040,548,1.2433,N,...,10,10,-1,11,-1,17,-1,20,N,NJ
2,4,2014-08-25,0,F,7,1.0006,0.0040,548,1.2433,N,...,15,18,-1,21,-1,11,-1,8,N,NJ
3,6,2013-04-15,0,J,10,0.9769,0.0004,"1,165",1.2665,N,...,6,5,-1,10,-1,9,-1,21,N,TX
4,8,2014-01-25,0,E,23,0.9472,0.0006,"1,487",1.3045,N,...,18,22,-1,10,-1,11,-1,12,N,IL


In [18]:
allstate.dtypes

QuoteConversion_Flag      int64
Field6                    int64
Field7                    int64
Field8                  float64
Field9                  float64
Field10                   int64
Field11                 float64
Field12                   int64
CoverageField1A           int64
CoverageField1B           int64
CoverageField2A           int64
CoverageField2B           int64
CoverageField3A           int64
CoverageField3B           int64
CoverageField4A           int64
CoverageField4B           int64
CoverageField5A           int64
CoverageField5B           int64
CoverageField6A           int64
CoverageField6B           int64
CoverageField8            int64
CoverageField9            int64
CoverageField11A          int64
CoverageField11B          int64
SalesField1A              int64
SalesField1B              int64
SalesField2A              int64
SalesField2B              int64
SalesField3               int64
SalesField4               int64
                         ...   
Geograph

In [16]:
allstate['Date'] = pd.to_datetime(pd.Series(allstate['Original_Quote_Date']))

In [17]:
allstate = allstate.drop(["Original_Quote_Date","QuoteNumber" ], axis=1)

In [18]:
allstate['weekday'] = allstate['Date'].dt.dayofweek
allstate['weekyear'] = allstate['Date'].dt.weekofyear
allstate['Year'] = allstate['Date'].dt.year
allstate['Month'] = allstate['Date'].dt.month

In [19]:
allstate = allstate.drop(["Date"], axis=1)

In [20]:
for f in allstate.columns:
    if allstate[f].dtype=='object':
        #print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(allstate[f].values))
        allstate[f] = lbl.transform(list(allstate[f].values))


In [21]:
allstate = allstate.fillna(-99)

In [14]:
a = ["x", "x", "x", "y", "y","z"]

In [15]:
lbl = preprocessing.LabelEncoder()
lbl.fit(list(a))

LabelEncoder()

In [16]:
lbl.transform(["x","y","z","z", "x", "y", "z"])

array([0, 1, 2, 2, 0, 1, 2])

In [22]:
target = allstate["QuoteConversion_Flag"]

In [23]:
allstate = allstate.drop(['QuoteConversion_Flag'], axis=1)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(allstate, target, test_size=0.25, random_state = 1234)

In [27]:
clf = xgb.XGBClassifier(n_estimators=100,
                        #eta = 0.01, 
                        learning_rate=0.01,
                        nthread=-1,
                        max_depth=12,
                        subsample=0.6,
                        colsample_bytree=0.6,
                       silent=False,
                       verbose = 10)


In [28]:
xgb_model = clf.fit(X_train, y_train, eval_metric="error")

In [29]:
pred = clf.predict( X_test )

/home/visa/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
pred

array([0, 0, 0, ..., 0, 1, 0])

In [30]:
confusion_matrix( y_test, pred)

array([[51944,  1010],
       [ 4155,  8080]])

In [48]:
(51944+8080)/(51944+1010+8080+4155.0)

0.9207688413689427

In [49]:
8080/(8080+1010.0)

0.8888888888888888

In [51]:
8080/(8080+4155.0)

0.6604004903964038

In [52]:
2*0.88*0.66/(0.88+0.66)

0.7542857142857142